# Globular Clusters

In this notebook, you will examine the stellar distribution of the Milky Way globular cluster, NGC 1904 (M79). By the end of this notebook, you will have set a minimum size scale for our Milky Way galaxy *and* a lower bound on the age of the Universe.

In [ ]:
# Standard python imports
%matplotlib inline
import matplotlib
matplotlib.rcParams['font.size'] = 20

from os.path import join,exists
import numpy as np
import pylab as plt

from astropy.io import fits 
import astropy.coordinates
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
def load_stars():
    """
    Load the DES stellar catalog from a given healpix. This applies a
    cut on EXTENDED_CLASS_MASH_SOF == 0 to select a high-purity
    stellar sample. See the DES DR1 paper for more details:
    https://arxiv.org/abs/1801.03181

    Parameters
    ----------
    hpx : NSIDE=32 healpix pixel to load catalog from.

    Returns
    -------
    stars : stellar catalog as numpy array

    """
    filepath = '/project2/astr13300/Week7/y3_gold_2_2_08668.fits'
    data = fits.open(filepath)[1].data
    stars = data[data['EXTENDED_CLASS_MASH_SOF'] == 0]
    return stars

In [ ]:
def load_isochrone(age, metallicity):
    """
    Load a synthetic isochrone in the DES bandpass from a library
    of PARSEC isochrones from Bressan et al. (2012):
    https://arxiv.org/abs/1208.4498

    Parameters
    ----------
    age         : isochrone age in Gyr
    metallicity : isocrhone metallicity (Z)

    Returns
    -------
    iso         : array of isochrone values
    """
    dirname = '/project2/astr13300/Week7/isochrones/des/bressan2012'
    filepath = join(dirname,f'iso_a{age:04.1f}_z{metallicity:0.5f}.dat')
    if not exists(filepath):
        msg  = "File not found: %s\n"%filepath
        msg += "Parameters out of range:\n"
        msg += "  1 Gyr <= Age <= 13.5 Gyr\n"
        msg += "  0.0010 <= Z <= 0.00010"
        raise Exception(msg)
    iso = np.genfromtxt(filepath,names=True,skip_header=13,comments='#')   
    return iso

In [ ]:
def plot_color_magnitude_diagram(stars=None):
    """Plot a color magnitude diagram of stars.
    
    Parameters
    ----------
    stars : stars to plot (or load if `None`)
    
    Returns
    -------
    color,mag : color and magnitude of stars
    """
    # Load the stars
    if stars is None: stars = load_stars()

    RA,DEC = 81.046, -24.525
    # Calculate angular separation
    gc = SkyCoord(RA*u.deg, DEC*u.deg )
    coords = SkyCoord(stars['RA']*u.deg,stars['DEC']*u.deg)
    sep = gc.separation(coords)

    # Select stars within 0.2 deg and with good magnitudes
    sel = sep < 0.2*u.deg
    sel &= stars['WAVG_MAG_PSF_G'] < 30
    sel &= stars['WAVG_MAG_PSF_R'] < 30
    data = stars[sel]

    mag    = data['WAVG_MAG_PSF_G']
    color  = (data['WAVG_MAG_PSF_G']-data['WAVG_MAG_PSF_R'])

    plt.figure(figsize=(8,6))
    plt.scatter(color,mag,alpha=0.2)
    plt.xlim(0.0,1.0)
    plt.ylim(24,17)
    plt.xlabel('g - r')
    plt.ylabel('g')
    return color,mag

In [ ]:
def plot_isochrone(age,metallicity,distance):
    """Load and plot an isochrone.

    Parameters
    ----------
    age : age of the isochrone (Gyr)
    metallicity : metallicity of the isochrone (mass fraction, unitless)
    distance : distance to stellar system (kpc)

    Returns
    -------
    None
    """
    isochrone = load_isochrone(age=age,metallicity=metallicity)
    distance_modulus = 5*np.log10(distance*100)
    plt.scatter(isochrone['g']-isochrone['r'],isochrone['g']+distance_modulus,c='r')

<div class="alert alert-info" style="color:black">
<h3>Exercise 1</h3>
<hr>
NGC 1904 (M79) is a Milky Way globular cluster located at RA,DEC = 81.046, -24.525. We are going to use data from DES DR1 to measure some of the properties of this cluster. 
<ol>
    <li> Use the <a href="http://legacysurvey.org/viewer?ra=81.046&dec=-24.525&zoom=8&layer=des-dr1">DECaLS Legacy Viewer</a> to examine a color image of this cluster (you may need to pan and zoom).</li>
    <li>What do you notice about the color of the most of the stars in the globular cluster relative to other stars and galaxies in this image?</li>
    <li>Astronomical data is never perfect. Identify at least one feature in the image that looks like an artifact of the instrument or data processing.</li>
</ol>
</div>

**Your answers here:**

<div class="alert alert-info" style="color:black">
<h3>Exercise 2</h3>
<hr>
To learn more about this globular cluster, we would like to make a <b>color-magnitude diagram</b> of its stars. In a color-magnitude diagram, stars are ordered by color (a proxy for temperature) on the x-axis and magnitude (a proxy for luminosity) on the y-axis. Run the code block below to create a color-magnitude diagram of stars around NGC 1904.
<ol>
    <li>What corner of the plot hosts faint red stars? (Careful, magnitudes are counter intuitive.)</li>
    <li>Where would the bright blue stars be? Why don't you see any?</li>
    <li>Where are the oldest stars on this figure? (Careful!)</li>
</ol>
</div>

In [ ]:
stars = load_stars()
color,mag = plot_color_magnitude_diagram(stars)

**Your answers here:**

<div class="alert alert-info" style="color:black">
<h3>Exercise 3</h3>
<hr>
The stars in NGC 1904 were created at very nearly the same time and have evolved together. However, stars evolved at different rates. More massive stars have shorter lifetimes (burn bright, die young!), while the faintest stars can live longer than the current age of the Universe. A snapshot of the population of stars at a given time is called an <b>isochrone</b> (from the Greek for "equal time"). By fitting an isochrone to the stellar distribution in NGC 1904 we can determine several properties of this cluster.
<ol>
    <li>Using the code below, overplot an isochrone on the color magnitude diagram.</li>
    <li>Adjust the distance (in kpc), age (in Gyr), and metallicity (unitless mass fraction) to find an isochrone that fits the distribution of stars in color-magnitude space.</li>
    <li>What are your best fit parameters? If the globular cluster was formed in our Galaxy, what does its distance tell you about the minimum size of our Galaxy? What does its age tell you about the minimum age of the Universe?</li>
</ol>
</div>

In [ ]:
color,mag = plot_color_magnitude_diagram(stars)
plot_isochrone(age=5, metallicity=0.0010, distance=10)

**Your answers here:**